In [ ]:
# --- PART 1: SETUP FOR COLAB T4 ---

# Define the working directory for Google Colab
WORK_DIR = "/content/ComfyUI"

import os

# 1. Clone the repository
if not os.path.exists(WORK_DIR):
    !git clone https://github.com/Imtiazul-Islam/ComfyUI {WORK_DIR}

# Navigate into the directory
%cd {WORK_DIR}

# Install requirements (quietly)
!pip install -q -r requirements.txt

# Install aria2 for fast downloads
!apt-get -y install -qq aria2

# 2. Download the models using aria2 from your Hugging Face Mirror
# Pointing to ImtiazulIslam/Z-Image-Turbo-FP8

print("Downloading Models (UNET, CLIP, VAE)...")

# Model 1: UNET (Diffusion Model) - FP8 Version
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ImtiazulIslam/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d {WORK_DIR}/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors

# Model 2: CLIP (Text Encoder) - Qwen/Lumina type
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ImtiazulIslam/Z-Image-Turbo-FP8/resolve/main/qwen_3_4b.safetensors -d {WORK_DIR}/models/clip -o qwen_3_4b.safetensors

# Model 3: VAE
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ImtiazulIslam/Z-Image-Turbo-FP8/resolve/main/ae.safetensors -d {WORK_DIR}/models/vae -o ae.safetensors

print("Setup Complete. Models loaded.")

In [ ]:
# --- PART 2: LOAD MODELS & UI (OPTIMIZED FOR COLAB) ---

%cd {WORK_DIR}

import os
import random
import time
import datetime
import sys

import torch
import numpy as np
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets

# Ensure ComfyUI modules are found
sys.path.insert(0, os.getcwd())

# Import ComfyUI Nodes
from nodes import NODE_CLASS_MAPPINGS

# Initialize Nodes
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

print("Loading Models into VRAM (this may take a moment)...")

# Load Models (Using fp8_e4m3fn_fast for T4 efficiency)
with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    # type="lumina2" is required for this specific CLIP model
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

print("Models Loaded Successfully. Initializing UI...")

# ==========================================
# GENERATION LOGIC
# ==========================================

@torch.inference_mode()
def run_generation(b):
    # UI Feedback
    gen_btn.disabled = True
    gen_btn.description = "Generating..."
    out_area.clear_output(wait=True)

    with out_area:
        print(f"Resolution: {width.value}x{height.value} | Steps: {steps.value} | Seed: {seed.value}")

        tmp_dir = f"{WORK_DIR}/output"
        os.makedirs(tmp_dir, exist_ok=True)

        # 1. Handle Seed
        current_seed = seed.value
        if current_seed == 0:
            random.seed(int(time.time()))
            current_seed = random.randint(0, 18446744073709551615)

        # 2. Encode Prompts
        positive = CLIPTextEncode.encode(clip, positive_prompt.value)[0]
        negative = CLIPTextEncode.encode(clip, negative_prompt.value)[0]

        # 3. Generate Latent
        latent_image = EmptyLatentImage.generate(width.value, height.value, batch_size=1)[0]

        # 4. Sample (Diffusion Process)
        samples = KSampler.sample(
            unet,
            current_seed,
            steps.value,
            cfg.value,
            sampler_name.value,
            scheduler.value,
            positive,
            negative,
            latent_image,
            denoise=denoise.value
        )[0]

        # 5. Decode VAE
        decoded = VAEDecode.decode(vae, samples)[0].detach()

        # 6. Save with Timestamp
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"z_turbo_{timestamp}.png"
        output_path = f"{tmp_dir}/{filename}"

        # Convert tensor to image and save
        img_array = np.array(decoded*255, dtype=np.uint8)[0]
        Image.fromarray(img_array).save(output_path)

        # 7. Display
        print(f"Done! Actual Seed: {current_seed}")
        display(Image.open(output_path))

    # Re-enable button
    gen_btn.disabled = False
    gen_btn.description = "Generate Image"

def update_dimensions(change):
    """Auto-update Width/Height based on Aspect Ratio selection"""
    ar = change['new']
    if ar == "Custom":
        return # Don't override if user selects Custom

    presets = {
        "1:1 (Square)": (1024, 1024),
        "16:9 (Landscape)": (1344, 768), # Slightly adjusted for better mult of 16
        "9:16 (Portrait)": (768, 1344),
        "4:3 (Photo)": (1152, 896),
        "3:2 (Classic)": (1216, 832),
        "21:9 (Ultrawide)": (1536, 640)
    }

    if ar in presets:
        w, h = presets[ar]
        width.value = w
        height.value = h

# ==========================================
# WIDGET DEFINITIONS
# ==========================================

style = {'description_width': 'initial'}

# Prompts
positive_prompt = widgets.Textarea(
    value="a cinematic photo of a cat, realistic, 8k, highly detailed",
    description='Positive:', style=style, layout=widgets.Layout(width='90%')
)
negative_prompt = widgets.Textarea(
    value="blurry, ugly, bad quality, distorted, low res",
    description='Negative:', style=style, layout=widgets.Layout(width='90%')
)

# Aspect Ratio & Dimensions
aspect_ratio = widgets.Dropdown(
    options=["1:1 (Square)", "16:9 (Landscape)", "9:16 (Portrait)", "4:3 (Photo)", "3:2 (Classic)", "21:9 (Ultrawide)", "Custom"],
    value="1:1 (Square)",
    description='Aspect Ratio:', style=style
)
aspect_ratio.observe(update_dimensions, names='value')

width = widgets.IntText(value=1024, description='Width:', style=style)
height = widgets.IntText(value=1024, description='Height:', style=style)

# Settings
steps = widgets.IntSlider(value=6, min=1, max=20, step=1, description='Steps:', style=style) # Z-Turbo works best with low steps (4-6)
cfg = widgets.FloatSlider(value=1.2, min=1.0, max=3.0, step=0.1, description='CFG Scale:', style=style)
seed = widgets.IntText(value=0, description='Seed (0=Random):', style=style)
sampler_name = widgets.Dropdown(
    options=['euler', 'euler_ancestral', 'dpmpp_2m', 'dpmpp_sde'],
    value='euler', description='Sampler:', style=style
)
scheduler = widgets.Dropdown(
    options=['simple', 'normal', 'karras', 'exponential', 'sgm_uniform'],
    value='simple', description='Scheduler:', style=style
)
denoise = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.05, description='Denoise:', style=style)

# Generate Button
gen_btn = widgets.Button(
    description='Generate Image',
    button_style='success',
    tooltip='Click to generate',
    icon='magic'
)
gen_btn.on_click(run_generation)

# Output Area
out_area = widgets.Output()

# ==========================================
# UI LAYOUT
# ==========================================

ui_header = widgets.HTML("<h2>Z-Image-Turbo Colab UI (T4 Optimized)</h2>")

# Left Column (Prompts)
left_col = widgets.VBox([
    ui_header,
    positive_prompt,
    negative_prompt,
    gen_btn,
    out_area
])

# Right Column (Settings)
right_col = widgets.VBox([
    widgets.HTML("<b>Image Settings</b>"),
    aspect_ratio,
    widgets.HBox([width, height]),
    widgets.HTML("<br><b>Sampling Settings</b>"),
    steps,
    cfg,
    seed,
    sampler_name,
    scheduler,
    denoise
])

# Main Layout
ui = widgets.HBox([left_col, right_col])
display(ui)